In [2]:
import sys
from pathlib import Path
import pandas as pd

REPO_ROOT = Path.cwd().parent.parent
BACKEND_DIR = REPO_ROOT / "backend"
sys.path.insert(0, str(BACKEND_DIR))
sys.path.insert(0, str(Path.cwd()))

from analytics.forecasting.prophet import ProphetForecaster
from _pool_common import (
    load_pool_data,
    backtest_one_step,
    compute_metrics,
    metrics_to_parquet,
    TEST_SIZE,
    MIN_TRAIN_PROPHET,
    ARTIFACTS_DIR,
)

In [3]:
stacked = load_pool_data()
print(stacked.groupby("symbol").size())
stacked.head(10)

symbol
AAPL       262
BTC-USD    262
ETH-USD    262
MSFT       262
NVDA       262
QQQ        262
SPY        262
dtype: int64


,timestamp,symbol,close
0,2021-02-22,AAPL,121.260002
1,2021-03-01,AAPL,121.419998
2,2021-03-08,AAPL,121.029999
3,2021-03-15,AAPL,119.989998
4,2021-03-22,AAPL,121.209999
5,2021-03-29,AAPL,123.000000
6,2021-04-05,AAPL,133.000000
7,2021-04-12,AAPL,134.160004
8,2021-04-19,AAPL,134.320007
9,2021-04-26,AAPL,131.460007


In [4]:
model_name = "prophet"
all_preds = []
for sym, grp in stacked.groupby("symbol"):
    prices = grp.set_index("timestamp")["close"].astype(float).dropna().sort_index()
    if len(prices) < TEST_SIZE + MIN_TRAIN_PROPHET:
        continue
    pred = backtest_one_step(
        prices, TEST_SIZE,
        model_factory=lambda: ProphetForecaster(confidence_level=0.95),
        min_train=MIN_TRAIN_PROPHET,
    )
    pred["symbol"] = sym
    all_preds.append(pred)

pred_prophet = pd.concat(all_preds, ignore_index=True)
print(pred_prophet.groupby("symbol").size())
pred_prophet.head()

c:\capstone_project_unfc\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
18:01:33 - cmdstanpy - INFO - Chain [1] start processing
18:01:33 - cmdstanpy - INFO - Chain [1] done processing
18:01:34 - cmdstanpy - INFO - Chain [1] start processing
18:01:34 - cmdstanpy - INFO - Chain [1] done processing
18:01:34 - cmdstanpy - INFO - Chain [1] start processing
18:01:34 - cmdstanpy - INFO - Chain [1] done processing
18:01:34 - cmdstanpy - INFO - Chain [1] start processing
18:01:34 - cmdstanpy - INFO - Chain [1] done processing
18:01:35 - cmdstanpy - INFO - Chain [1] start processing
18:01:35 - cmdstanpy - INFO - Chain [1] done processing
18:01:35 - cmdstanpy - INFO - Chain [1] start processing
18:01:35 - cmdstanpy - INFO - Chain [1] done processing
18:01:35 - c

symbol
AAPL       30
BTC-USD    30
ETH-USD    30
MSFT       30
NVDA       30
QQQ        30
SPY        30
dtype: int64


,timestamp,y_true,y_pred,symbol
0,2025-08-04,229.350006,227.5549,AAPL
1,2025-08-11,231.589996,230.5641,AAPL
2,2025-08-18,227.759995,234.4034,AAPL
3,2025-08-25,232.139999,230.2365,AAPL
4,2025-09-01,239.690002,229.5787,AAPL


In [5]:
metrics_rows = []
for sym in pred_prophet["symbol"].unique():
    sub = pred_prophet[pred_prophet["symbol"] == sym]
    m = compute_metrics(sub)
    metrics_rows.append({"model": model_name, "symbol": sym, **m})
m_overall = compute_metrics(pred_prophet)
metrics_rows.append({"model": model_name, "symbol": "overall", **m_overall})

metrics_df = pd.DataFrame(metrics_rows)
print(metrics_df.to_string())
metrics_to_parquet(metrics_rows, ARTIFACTS_DIR / "metrics_prophet_pool.parquet")
print("Saved:", ARTIFACTS_DIR / "metrics_prophet_pool.parquet")

     model   symbol           MAE          RMSE     MAPE_%
0  prophet     AAPL     15.397031     18.404163   5.844861
1  prophet  BTC-USD  22038.800661  26618.561264  25.995763
2  prophet  ETH-USD    926.280578   1012.195513  28.834370
3  prophet     MSFT     41.723848     46.972947   8.949213
4  prophet     NVDA      9.645319     12.404388   5.267705
5  prophet      QQQ     22.340317     26.367803   3.706778
6  prophet      SPY     19.572718     23.267854   2.937659
7  prophet  overall   3296.251496  10068.169639  11.648050
Saved: C:\capstone_project_unfc\model\experiments-pool\artifacts\metrics_prophet_pool.parquet
